In [1]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, read_data
import polars as pl
from datetime import date

In [2]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

In [3]:
RPL.df

code,date,open,high,low,close,volume,accumulated_split_rate
i64,date,f64,f64,f64,f64,f64,f64
1301,2009-01-05,2100.0,2120.0,2060.0,2070.0,11500.0,10.0
1301,2009-01-06,2070.0,2090.0,2050.0,2070.0,27300.0,10.0
1301,2009-01-07,2050.0,2080.0,2050.0,2070.0,26900.0,10.0
1301,2009-01-08,2050.0,2070.0,2040.0,2060.0,19900.0,10.0
1301,2009-01-09,2070.0,2080.0,2050.0,2050.0,24300.0,10.0
…,…,…,…,…,…,…,…
9997,2024-11-25,740.0,740.0,734.0,737.0,169400.0,1.0
9997,2024-11-26,738.0,738.0,732.0,736.0,67800.0,1.0
9997,2024-11-27,736.0,736.0,725.0,732.0,70600.0,1.0


In [ ]:
# 開発


In [4]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

code = 1301

# codeで指定した証券コードの銘柄の指定した日付における最新の終値の株価を、(取引日, 株価)のタプルで返す
# def get_latest_dealingdate_and_price(self, code: int, valuation_date: date = date.today()) -> tuple:
tup = RPL.get_latest_dealingdate_and_price(code)
tup

(datetime.date(2024, 11, 29), 3910.0)

In [5]:
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RPL = PricelistPl(df)

# colで指定した列のterm日の移動平均列を、25日移動平均であれば、ma25の
# ような列名(maの後ろに移動平均の日数)で追加する。
# termで指定した日数での移動平均が計算できない初期のレコードは、dropされてなくなる
# 全データで実施すると、かなりメモリを消費するので、200日移動平均などを取得する場合は、
# PricelistPl(filename).dfをfilterしてから実施しないとメモリが足りなくなるかもしれない。
# メモリが不足して実行プロセスがダウンした場合は、例外も出力されない。
# def with_columns_moving_average(self, term, col="close"):
RPL.with_columns_moving_average(25)
RPL.df

code,date,open,high,low,close,volume,accumulated_split_rate,ma25
i64,date,f64,f64,f64,f64,f64,f64,f64
1301,2009-02-09,1910.0,1930.0,1900.0,1900.0,10900.0,10.0,1984.8
1301,2009-02-10,1910.0,1950.0,1910.0,1930.0,8900.0,10.0,1979.2
1301,2009-02-12,1900.0,1920.0,1890.0,1900.0,12800.0,10.0,1972.4
1301,2009-02-13,1900.0,1930.0,1900.0,1930.0,8800.0,10.0,1966.8
1301,2009-02-16,1920.0,1940.0,1910.0,1940.0,9600.0,10.0,1962.0
…,…,…,…,…,…,…,…,…
9997,2024-11-25,740.0,740.0,734.0,737.0,169400.0,1.0,722.84
9997,2024-11-26,738.0,738.0,732.0,736.0,67800.0,1.0,723.48
9997,2024-11-27,736.0,736.0,725.0,732.0,70600.0,1.0,723.84
